# Model-based Collaborative filtering with GA4 Dallas Free Press dataset

* Dataset: GA4 DFP
* CF-type: Model
* Implicit/Explicit: Implicit
* User-User/Item-Item: Does not apply

## Approach

In this notebook, we're going to build up a model-based collaborative filter. 

We're going to start by building a model with a single piece of user feedback. 

Once that's established, we'll start incorporating multiple sources of feedback using feature aware matrix factorization. More details [here](https://docs.google.com/document/d/1mO94wAjdVFDRKSQo7JYVkd_NsiBDkHxhfyj7u_uZY9Q/edit?tab=t.0#heading=h.yevokmfzxk7). 